<a href="https://colab.research.google.com/github/JCaballerot/Decision_Tree_Learning/blob/main/1.%20Classification_trees/%C3%81rboles_de_Clasificaci%C3%B3n_en_Python_con_Databricks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://www.ctic.uni.edu.pe/wp-content/uploads/2022/04/588px-x-348px-web-1.png" alt="HTML5 Icon" width="900" height="350" >


<h1 align=center><font size = 5>Laboratorio: Árboles de Clasificación en Python con Databricks
</font></h1>


## Objetivos
- Aplicar técnicas de preprocesamiento de datos categóricos utilizando Target Encoding.
- Balancear un conjunto de datos desbalanceado utilizando class_weight.
- Manejar los valores faltantes de forma adecuada y aplicar el tratamiento correspondiente en los conjuntos de entrenamiento y prueba.
- Construir, explorar y visualizar un árbol de clasificación.
- Optimizar hiperparámetros del árbol utilizando Grid Search.
- Calcular y explicar las métricas principales: Accuracy y Gini.

---

## Tabla de Contenidos

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
    
1. <a href="#item31">Introducción</a>  

2. <a href="#item32">Preprocesamiento de Datos</a>  
    - Carga y revisión de los datos
    - Muestreo de datos
    - Tratamiento de valores faltantes
    - Encoding de datos categóricos (Target Encoding)

3. <a href="#item33">Balanceo de datos</a>  
    - Reflexión sobre el desbalanceo y uso de class_weight

4. <a href="#item33">Construcción y Visualización del Árbol de Clasificación</a>  
    - Creación y visualización gráfica del árbol

5. <a href="#item34">Optimización del Modelo</a>  
    - Predicciones y evaluación del modelo
    - Uso de Grid Search para optimizar hiperparámetros

6. <a href="#item34">Evaluación del modelo</a>  
    - Cálculo y explicación de Accuracy y Gini




## 1. Introducción


En este laboratorio, aprenderemos a construir un árbol de clasificación utilizando técnicas avanzadas de preprocesamiento y optimización. La base de datos utilizada será HM_FLUJOS_fad_mvp2.csv. El objetivo será predecir la variable DEF12 (indicador de incumplimiento) basándonos en una serie de variables numéricas y categóricas.

## 2. Preprocesamiento de Datos


**Carga del Dataset**

Primero, cargamos el dataset y revisamos su estructura.



In [ ]:
import pandas as pd

# Cargar el dataset
pddf = pd.read_csv('/Workspace/Users/jcaballerot@bcp.com.pe/2024Q3 - Decision Trees/data/HM_FLUJOS_fad_mvp2.csv', index_col=0)

# Mostrar las primeras filas del dataset
pddf.head()


**Muestreo de Datos**

Dividimos los datos en conjuntos de entrenamiento y prueba, donde DEF12 será la variable objetivo y las variables numéricas y categóricas se utilizarán como características.

In [ ]:
from sklearn.model_selection import train_test_split

# Definir las variables numéricas y categóricas
features = [
 'MONTO', 'MTO_DEURCCVIG', 'MTO_DEURCCVIG_AVGU12', 'R_MTODEURCCVIG_AVGU06_AVGU12', 'MTO_ABODEP',
 'MTO_ABODEP_AVGU12', 'FRQ_PASPM_GT1000U06', 'FRQ_PASPM_GT1000U12', 'MTO_PAS_AVGU03', 'MTO_PAS_AVGU06',
 'MTO_PAS_AVGU12', 'R_PASPM_U03U06', 'R_PASPM_U06U12', 'MTO_PASFM', 'MTO_PASFM_AVGU12',
 'R_MTOAPROB_VTAEST', 'R_MTOAPROB_PASAVGU12', 'R_MTOAPROB_ABODEPAVGU12', 'R_MTODEUAVGU12_VTAEST',
 'R_MTODEUAVGU12_PASAU12', 'R_MTODEUAVGU12_ABODEPAU12', 'MAX_CTDDIAATRASO', 'MAX_CTDDIAATRASO_U06',
 'MAX_CTDDIAATRASO_NOBCP'
]




In [ ]:
categorical_columns = ['CAT_SUNAT_SUNATRES']

# Definir las características (X) y la variable objetivo (y)
X = pddf[features + categorical_columns]
y = pddf['DEF12']


In [ ]:

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=123)

**Tratamiento de Valores Faltantes**

Los valores faltantes pueden afectar el rendimiento del modelo, por lo que debemos imputarlos de forma adecuada. Usaremos la media para las variables numéricas y aplicaremos el mismo tratamiento en el conjunto de prueba.

In [ ]:
# Rellenar valores faltantes con la media en las variables numéricas
X_train[features] = X_train[features].fillna(X_train[features].mean())
X_test[features] = X_test[features].fillna(X_train[features].mean())


**Target Encoding para Variables Categóricas**

Aplicamos Target Encoding a la variable categórica CAT_SUNAT_SUNATRES utilizando la relación con la variable objetivo DEF12.

In [ ]:
from category_encoders import TargetEncoder

# Aplicar Target Encoding
encoder = TargetEncoder(cols=categorical_columns)
X_train = encoder.fit_transform(X_train, y_train)
X_test = encoder.transform(X_test)

## 3. Balanceo de Datos



Al observar el dataset, podemos encontrar un desbalance en las clases de DEF12. Esto significa que, sin el tratamiento adecuado, el modelo podría inclinarse hacia la predicción de la clase mayoritaria. Para mitigar este problema, utilizaremos la técnica de class_weight='balanced' en el modelo de clasificación, que ajusta los pesos de las clases de acuerdo a su frecuencia en el dataset.


In [ ]:
# Verificar el desbalanceo en el conjunto de entrenamiento
print(y_train.value_counts(normalize=True))

In [ ]:
import seaborn as sns
sns.set(style="whitegrid", color_codes = True)
sns.set(rc={'figure.figsize':(8,4)})

sns.countplot(x = 'DEF12', data = pddf, palette = 'hls')


## 4. Construcción y Visualización del Árbol de Clasificación



Ahora entrenamos el árbol de clasificación, aplicando class_weight para balancear las clases y utilizamos la visualización del árbol.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Definir el modelo con class_weight balanceado
clf = DecisionTreeClassifier(max_depth=5,
                             class_weight='balanced',
                             random_state=123)

# Entrenar el modelo
clf.fit(X_train, y_train)


In [ ]:
from sklearn import tree
import matplotlib.pyplot as plt

# Ajustar el tamaño de la figura
plt.figure(figsize=(60, 20))

# Graficar el árbol de decisión
tree.plot_tree(clf,
               feature_names=features,
               filled=True,
               fontsize=20,  # Ajustar el tamaño de la fuente aquí
               node_ids=True)  # Opcional: añadir IDs a los nodos para referencia

# Mostrar el gráfico
plt.show()

## 5. Optimización del Modelo (Grid Search)


Realizamos un Grid Search para optimizar los hiperparámetros del modelo, buscando la mejor combinación de profundidad del árbol y número mínimo de muestras por hoja.

In [ ]:
from sklearn.model_selection import GridSearchCV

# Definir los hiperparámetros a buscar
param_grid = {
    'max_depth': [3, 5, 7, 10],
    'min_samples_leaf': [0.01, 0.05, 0.1],
    'criterion': ['gini', 'entropy']
}

# Realizar el Grid Search con validación cruzada
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='roc_auc', n_jobs=-1, verbose=1, return_train_score=True)
grid_search.fit(X_train, y_train)


# Ver los mejores parámetros
print("Mejores parámetros encontrados: ", grid_search.best_params_)


In [ ]:
# Obtener resultados en forma de DataFrame
results = pd.DataFrame(grid_search.cv_results_)

In [ ]:
import pandas as pd
from sklearn.metrics import roc_auc_score

# Obtener los resultados del Grid Search
results = pd.DataFrame(grid_search.cv_results_)

# Calcular el Gini (2 * AUC - 1) para train y test
results['gini_train'] = 2 * results['mean_train_score'] - 1
results['gini_test'] = 2 * results['mean_test_score'] - 1

# Seleccionar las columnas de interés
results_df = results[['param_max_depth', 'param_min_samples_leaf', 'param_min_samples_split',
                      'param_criterion', 'gini_train', 'gini_test']]

# Mostrar la tabla de resultados
results_df.head()


In [ ]:
import matplotlib.pyplot as plt

# Gráfico de Gini vs max_depth
plt.figure(figsize=(10, 6))
for criterion in results_df['param_criterion'].unique():
    # Filtrar por criterio
    subset = results_df[results_df['param_criterion'] == criterion]

    plt.plot(subset['param_max_depth'], subset['gini_train'], label=f'Train {criterion}', marker='o')
    plt.plot(subset['param_max_depth'], subset['gini_test'], label=f'Test {criterion}', marker='x')

# Etiquetas y título
plt.xlabel('max_depth')
plt.ylabel('Gini')
plt.title('Gini vs max_depth para Train y Test')
plt.legend()
plt.grid(True)
plt.show()


## 6. Evaluación del Modelo


**Accuracy**

La métrica Accuracy mide la proporción de predicciones correctas.

In [ ]:
from sklearn.metrics import accuracy_score

# Predicciones en el conjunto de prueba
y_pred = grid_search.predict(X_test)

# Calcular la Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


**Gini**

El índice de Gini se calcula como 2 * AUC - 1, y mide la capacidad del modelo para diferenciar entre clases.

In [ ]:
from sklearn.metrics import roc_auc_score

# Calcular AUC
auc = roc_auc_score(y_test, grid_search.predict_proba(X_test)[:,1])

# Calcular Gini
gini = 2 * auc - 1
print("Gini:", gini)


## Conclusiones

En este laboratorio, hemos cubierto los pasos necesarios para construir un árbol de clasificación, desde el preprocesamiento con Target Encoding y el manejo de valores faltantes, hasta la optimización del modelo y la evaluación con métricas clave como Accuracy y Gini. Además, discutimos cómo el desbalanceo puede afectar los resultados y cómo mitigarlo mediante el uso de class_weight.

---

# Gracias por completar este laboratorio!

---